<a href="https://colab.research.google.com/github/YianKim/2022_PythonStudy_face_emotion_recognition/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [254]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리 불러오기


In [255]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [256]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

In [257]:
train_data = data[range(20000)].reshape([20000,28,28,1])
train_labels = labels[range(20000)].reshape([20000,1])

In [258]:
unlab_data = data[range(20000,60000)].reshape([40000,28,28,1])
unlab_labels = labels[range(20000,60000)].reshape([40000,1])

In [259]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

## CNN

In [260]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [261]:
#######HYPERPARAMATERS###########
epochs = 20
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/20
219/219 [==============================] - 4s 15ms/step - loss: 1.8003 - accuracy: 0.6882 - val_loss: 0.6811 - val_accuracy: 0.7640
Epoch 2/20
219/219 [==============================] - 3s 13ms/step - loss: 0.5798 - accuracy: 0.7969 - val_loss: 0.5702 - val_accuracy: 0.7935
Epoch 3/20
219/219 [==============================] - 3s 12ms/step - loss: 0.4958 - accuracy: 0.8213 - val_loss: 0.5419 - val_accuracy: 0.8093
Epoch 4/20
219/219 [==============================] - 3s 12ms/step - loss: 0.4454 - accuracy: 0.8377 - val_loss: 0.5123 - val_accuracy: 0.8168
Epoch 5/20
219/219 [==============================] - 3s 12ms/step - loss: 0.4117 - accuracy: 0.8468 - val_loss: 0.4998 - val_accuracy: 0.8270
Epoch 6/20
219/219 [==============================] - 3s 11ms/step - loss: 0.3818 - accuracy: 0.8562 - val_loss: 0.5125 - val_accuracy: 0.8332
Epoch 7/20
219/219 [==============================] - 3s 12ms/step - loss: 0.3645 - accuracy: 0.8660 - val_loss: 0.5029 - val_accuracy: 0.8338

In [262]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [263]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [264]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(test_labels, predicted_test_labels))

In [265]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval() #0.89xx

0.8907

# 라벨링

## 라벨링; 1

In [266]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [267]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(np.argmax(temp2))
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 40000/40000 [00:07<00:00, 5478.92it/s]


In [268]:
lowvars = []
ind = 0 

for i in Vars:
  if i < 1e-5:
    lowvars.append(ind)
  ind += 1

In [269]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [270]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정 : 0.90x
randomindx = random.sample(range(40000), len(lowvars))
accuracy_score(np.array(Outs)[randomindx], unlab_labels[randomindx])

0.8907412920512057

In [271]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 0.99x
accuracy_score(np.array(Outs)[lowvars], unlab_labels[lowvars])

0.9993053488141312

In [272]:
train_data = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels = np.concatenate([train_labels, unlab_labels[lowvars]], axis=0)

In [273]:
shufindx = random.sample(range(train_data.shape[0]),train_data.shape[0])
train_data = train_data[shufindx]
train_labels = train_labels[shufindx]

In [274]:
unlab_data = unlab_data[highvars]
unlab_labels = unlab_labels[highvars]

## 모델링; 1

In [275]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [276]:
#######HYPERPARAMATERS###########
epochs = 20
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/20
329/329 [==============================] - 5s 12ms/step - loss: 1.2070 - accuracy: 0.7763 - val_loss: 0.4562 - val_accuracy: 0.8395
Epoch 2/20
329/329 [==============================] - 4s 12ms/step - loss: 0.4084 - accuracy: 0.8579 - val_loss: 0.3927 - val_accuracy: 0.8595
Epoch 3/20
329/329 [==============================] - 4s 11ms/step - loss: 0.3484 - accuracy: 0.8734 - val_loss: 0.3650 - val_accuracy: 0.8713
Epoch 4/20
329/329 [==============================] - 4s 11ms/step - loss: 0.3184 - accuracy: 0.8831 - val_loss: 0.3615 - val_accuracy: 0.8753
Epoch 5/20
329/329 [==============================] - 4s 11ms/step - loss: 0.2879 - accuracy: 0.8965 - val_loss: 0.3254 - val_accuracy: 0.8838
Epoch 6/20
329/329 [==============================] - 4s 12ms/step - loss: 0.2659 - accuracy: 0.9025 - val_loss: 0.3236 - val_accuracy: 0.8880
Epoch 7/20
329/329 [==============================] - 4s 12ms/step - loss: 0.2531 - accuracy: 0.9050 - val_loss: 0.3169 - val_accuracy: 0.8935

In [277]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [278]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval() #0.89xx

0.8856

## 라벨링; 2

In [279]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [280]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(np.argmax(temp2))
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 29923/29923 [00:05<00:00, 5330.33it/s]


In [281]:
lowvars = []
ind = 0 

for i in Vars:
  if i < 1e-5:
    lowvars.append(ind)
  ind += 1

In [282]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [283]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정 : 0.90x
randomindx = random.sample(range(unlab_data.shape[0]), len(lowvars))
accuracy_score(np.array(Outs)[randomindx], unlab_labels[randomindx])

0.8642512077294686

In [284]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 0.99x
accuracy_score(np.array(Outs)[lowvars], unlab_labels[lowvars])

0.998792270531401

In [285]:
len(highvars)

25783

In [286]:
train_data = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels = np.concatenate([train_labels, unlab_labels[lowvars]], axis=0)

In [287]:
unlab_data = unlab_data[highvars]
unlab_labels = unlab_labels[highvars]

In [288]:
shufindx = random.sample(range(train_data.shape[0]),train_data.shape[0])
train_data = train_data[shufindx]
train_labels = train_labels[shufindx]

In [289]:
unlab_data.shape

(25783, 28, 28, 1)

## 모델링; 2

In [290]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [291]:
#######HYPERPARAMATERS###########
epochs = 20
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/20
375/375 [==============================] - 5s 12ms/step - loss: 1.2771 - accuracy: 0.8043 - val_loss: 0.3862 - val_accuracy: 0.8641
Epoch 2/20
375/375 [==============================] - 4s 11ms/step - loss: 0.3550 - accuracy: 0.8770 - val_loss: 0.3648 - val_accuracy: 0.8788
Epoch 3/20
375/375 [==============================] - 4s 11ms/step - loss: 0.3170 - accuracy: 0.8881 - val_loss: 0.3155 - val_accuracy: 0.8868
Epoch 4/20
375/375 [==============================] - 4s 11ms/step - loss: 0.2870 - accuracy: 0.8971 - val_loss: 0.3269 - val_accuracy: 0.8920
Epoch 5/20
375/375 [==============================] - 5s 13ms/step - loss: 0.2666 - accuracy: 0.9023 - val_loss: 0.3088 - val_accuracy: 0.8916
Epoch 6/20
375/375 [==============================] - 5s 13ms/step - loss: 0.2466 - accuracy: 0.9094 - val_loss: 0.2842 - val_accuracy: 0.9023
Epoch 7/20
375/375 [==============================] - 5s 13ms/step - loss: 0.2372 - accuracy: 0.9139 - val_loss: 0.2835 - val_accuracy: 0.9031

In [292]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')

In [293]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')
model_eval() #0.89xx

0.8817